<a href="https://colab.research.google.com/github/VincentCloud/COMP550-final-project/blob/main/Bert_classification_hypothesis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install -q -U tensorflow-text
!pip install -q tf-models-official


     |████████████████████████████████| 3.4 MB 8.7 MB/s 
     |████████████████████████████████| 61 kB 487 kB/s 
     |████████████████████████████████| 3.3 MB 33.3 MB/s 
     |████████████████████████████████| 895 kB 47.3 MB/s 
     |████████████████████████████████| 596 kB 44.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 4.9 MB 8.2 MB/s 
     |████████████████████████████████| 1.8 MB 8.3 MB/s 
     |████████████████████████████████| 352 kB 55.2 MB/s 
     |████████████████████████████████| 1.2 MB 52.3 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 47.6 MB 1.4 MB/s 
     |████████████████████████████████| 1.1 MB 43.3 MB/s 
     |████████████████████████████████| 90 kB 8.7 MB/s 
     |████████████████████████████████| 99 kB 7.8 MB/s 
     |████████████████████████████████| 213 kB 38.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt


In [ ]:
def ingest(file_path):
    data = pd.read_csv(file_path)
    data.drop(['movie_ids', 'ratings'], axis=1, inplace=True)
    data = data[data.review_texts.isnull() == False]
    data['ratings_binary'] = data['ratings_binary'].apply(lambda e : 1 if e == 'p' else 0)
    # data['review_texts'] = data['review_texts'].map(int)
    # data = data[data['review_texts'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop(['index',"Unnamed: 0"], axis=1, inplace=True)
    print ('dataset loaded with shape', data.shape)
    return data

def build_classifier_model():
  tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
  tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

def build_optimizer(train_size, epochs = 5):
  steps_per_epoch = train_size
  num_train_steps = steps_per_epoch * epochs
  num_warmup_steps = int(0.1*num_train_steps)

  init_lr = 3e-5
  optimizer = optimization.create_optimizer(init_lr=init_lr,
                                            num_train_steps=num_train_steps,
                                            num_warmup_steps=num_warmup_steps,
                                            optimizer_type='adamw')
  return optimizer


In [ ]:
def run(file_path, rs):
  data = ingest(file_path)
  audience_data = data.drop(["sources"], axis=1)
  audience_data = audience_data.sample(frac=1).reset_index(drop=True)
  aud_x_train, aud_x_test, aud_y_train, aud_y_test = train_test_split(np.array(audience_data.review_texts),
                                                      np.array(audience_data.ratings_binary), test_size=0.3)
  aud_x_vali, aud_x_test, aud_y_vali, aud_y_test = train_test_split(aud_x_test,
                                                      aud_y_test, test_size=0.6, random_state=rs)

  # bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 



  aud_classifier_model = build_classifier_model()
  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  metrics = tf.metrics.BinaryAccuracy()



  epochs = 5

  aud_optimizer = build_optimizer(len(aud_x_train), epochs)


  aud_classifier_model.compile(optimizer=aud_optimizer,
                          loss=loss,
                          metrics=metrics)

  aud_history = aud_classifier_model.fit(x=aud_x_train, y=aud_y_train,
                                validation_data=(aud_x_vali, aud_y_vali),
                                epochs=epochs)

  loss, accuracy = aud_classifier_model.evaluate(x=aud_x_test, y=aud_y_test)

  print(f'Loss: {loss}')
  print(f'Accuracy: {accuracy}')

In [ ]:
data_files = ['longer_audience.csv', 'longer_critics.csv', 'shorter_audience.csv', 'shorter_critics.csv', 'critics_half.csv', 'audience_half.csv']


In [ ]:
file_path = f'/content/drive/MyDrive/dataset/{data_files[0]}'
run(file_path, 10)

INFO:absl:Using /tmp/tfhub_modules to cache modules.


dataset loaded with shape (2265, 4)


INFO:absl:using Adamw optimizer
INFO:absl:gradient_clip_norm=1.000000


Epoch 1/10
50/50 [==============================] - 432s 8s/step - loss: 0.6760 - binary_accuracy: 0.5514 - val_loss: 0.6422 - val_binary_accuracy: 0.6066
Epoch 2/10
50/50 [==============================] - 435s 9s/step - loss: 0.6544 - binary_accuracy: 0.5584 - val_loss: 0.6137 - val_binary_accuracy: 0.6801
Epoch 3/10
50/50 [==============================] - 441s 9s/step - loss: 0.6068 - binary_accuracy: 0.6467 - val_loss: 0.5864 - val_binary_accuracy: 0.6728
Epoch 4/10
50/50 [==============================] - 443s 9s/step - loss: 0.5711 - binary_accuracy: 0.6883 - val_loss: 0.5488 - val_binary_accuracy: 0.7279
Epoch 5/10
50/50 [==============================] - 442s 9s/step - loss: 0.5179 - binary_accuracy: 0.7445 - val_loss: 0.5342 - val_binary_accuracy: 0.6875
Epoch 6/10
50/50 [==============================] - 442s 9s/step - loss: 0.4606 - binary_accuracy: 0.7811 - val_loss: 0.5003 - val_binary_accuracy: 0.7316
Epoch 7/10
50/50 [==============================] - 443s 9s/step - los

In [ ]:
file_path = f'/content/drive/MyDrive/dataset/{data_files[1]}'
run(file_path, 15)

dataset loaded with shape (4275, 4)


INFO:absl:using Adamw optimizer
INFO:absl:gradient_clip_norm=1.000000


Epoch 1/5
94/94 [==============================] - 26s 192ms/step - loss: 0.7404 - binary_accuracy: 0.5963 - val_loss: 0.6183 - val_binary_accuracy: 0.5965
Epoch 2/5
94/94 [==============================] - 17s 186ms/step - loss: 0.6522 - binary_accuracy: 0.5812 - val_loss: 0.5717 - val_binary_accuracy: 0.6803
Epoch 3/5
94/94 [==============================] - 17s 185ms/step - loss: 0.6011 - binary_accuracy: 0.6430 - val_loss: 0.5243 - val_binary_accuracy: 0.7115
Epoch 4/5
94/94 [==============================] - 17s 185ms/step - loss: 0.5346 - binary_accuracy: 0.7052 - val_loss: 0.4749 - val_binary_accuracy: 0.7836
Epoch 5/5
25/25 [==============================] - 2s 98ms/step - loss: 0.4638 - binary_accuracy: 0.7883
Loss: 0.4638107120990753
Accuracy: 0.7883116602897644


In [ ]:
file_path = f'/content/drive/MyDrive/dataset/{data_files[2]}'
run(file_path, 15)

dataset loaded with shape (2268, 4)


INFO:absl:using Adamw optimizer
INFO:absl:gradient_clip_norm=1.000000


Epoch 1/5
50/50 [==============================] - 19s 197ms/step - loss: 0.6318 - binary_accuracy: 0.5684 - val_loss: 0.6330 - val_binary_accuracy: 0.6360
Epoch 2/5
50/50 [==============================] - 9s 184ms/step - loss: 0.5803 - binary_accuracy: 0.6818 - val_loss: 0.5747 - val_binary_accuracy: 0.6949
Epoch 3/5
50/50 [==============================] - 9s 184ms/step - loss: 0.5222 - binary_accuracy: 0.7328 - val_loss: 0.5193 - val_binary_accuracy: 0.7353
Epoch 4/5
50/50 [==============================] - 9s 185ms/step - loss: 0.4691 - binary_accuracy: 0.7738 - val_loss: 0.4614 - val_binary_accuracy: 0.7868
Epoch 5/5
13/13 [==============================] - 1s 101ms/step - loss: 0.4284 - binary_accuracy: 0.7531
Loss: 0.4284345507621765
Accuracy: 0.7530562281608582


In [ ]:
file_path = f'/content/drive/MyDrive/dataset/{data_files[3]}'
run(file_path, 16)

dataset loaded with shape (4631, 4)


INFO:absl:using Adamw optimizer
INFO:absl:gradient_clip_norm=1.000000


Epoch 1/5
102/102 [==============================] - 27s 190ms/step - loss: 0.7285 - binary_accuracy: 0.5594 - val_loss: 0.6633 - val_binary_accuracy: 0.5701
Epoch 2/5
102/102 [==============================] - 19s 183ms/step - loss: 0.6499 - binary_accuracy: 0.5869 - val_loss: 0.5981 - val_binary_accuracy: 0.6709
Epoch 3/5
102/102 [==============================] - 19s 183ms/step - loss: 0.5845 - binary_accuracy: 0.6770 - val_loss: 0.5325 - val_binary_accuracy: 0.7014
Epoch 4/5
102/102 [==============================] - 19s 183ms/step - loss: 0.5278 - binary_accuracy: 0.7192 - val_loss: 0.4767 - val_binary_accuracy: 0.7626
Epoch 5/5
27/27 [==============================] - 3s 96ms/step - loss: 0.4539 - binary_accuracy: 0.7710
Loss: 0.4539247155189514
Accuracy: 0.7709832191467285


In [ ]:
file_path = f'/content/drive/MyDrive/dataset/{data_files[4]}'
run(file_path, 12)

dataset loaded with shape (8906, 4)


INFO:absl:using Adamw optimizer
INFO:absl:gradient_clip_norm=1.000000


Epoch 1/5
195/195 [==============================] - 45s 189ms/step - loss: 0.7050 - binary_accuracy: 0.4676 - val_loss: 0.6482 - val_binary_accuracy: 0.5506
Epoch 2/5
195/195 [==============================] - 36s 186ms/step - loss: 0.6261 - binary_accuracy: 0.6131 - val_loss: 0.5647 - val_binary_accuracy: 0.6863
Epoch 3/5
195/195 [==============================] - 36s 187ms/step - loss: 0.5368 - binary_accuracy: 0.7114 - val_loss: 0.4955 - val_binary_accuracy: 0.7444
Epoch 4/5
195/195 [==============================] - 36s 186ms/step - loss: 0.4733 - binary_accuracy: 0.7543 - val_loss: 0.4651 - val_binary_accuracy: 0.7734
Epoch 5/5
51/51 [==============================] - 5s 101ms/step - loss: 0.4561 - binary_accuracy: 0.7700
Loss: 0.4560893774032593
Accuracy: 0.7699501514434814


In [ ]:
file_path = f'/content/drive/MyDrive/dataset/{data_files[5]}'
run(file_path, 4)

dataset loaded with shape (4533, 4)


INFO:absl:using Adamw optimizer
INFO:absl:gradient_clip_norm=1.000000


Epoch 1/5
100/100 [==============================] - 27s 193ms/step - loss: 0.6330 - binary_accuracy: 0.5626 - val_loss: 0.5419 - val_binary_accuracy: 0.7151
Epoch 2/5
100/100 [==============================] - 19s 186ms/step - loss: 0.5498 - binary_accuracy: 0.7025 - val_loss: 0.4535 - val_binary_accuracy: 0.7904
Epoch 3/5
100/100 [==============================] - 20s 195ms/step - loss: 0.4699 - binary_accuracy: 0.7649 - val_loss: 0.3835 - val_binary_accuracy: 0.8180
Epoch 4/5
100/100 [==============================] - 19s 187ms/step - loss: 0.4149 - binary_accuracy: 0.7974 - val_loss: 0.3270 - val_binary_accuracy: 0.8548
Epoch 5/5
26/26 [==============================] - 3s 101ms/step - loss: 0.3674 - binary_accuracy: 0.8137
Loss: 0.3673551678657532
Accuracy: 0.813725471496582
